<a href="https://www.kaggle.com/code/aleksandrmorozov123/video-detection-algorithms?scriptVersionId=176698339" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aggressive-behavior-video-classification/aggressive_behavior.csv
/kaggle/input/aggressive-behavior-video-classification/files/aggressive/3.mp4
/kaggle/input/aggressive-behavior-video-classification/files/aggressive/1.mp4
/kaggle/input/aggressive-behavior-video-classification/files/aggressive/4.mp4
/kaggle/input/aggressive-behavior-video-classification/files/aggressive/0.mp4
/kaggle/input/aggressive-behavior-video-classification/files/aggressive/2.mp4
/kaggle/input/aggressive-behavior-video-classification/files/non_aggressive/5.mp4
/kaggle/input/aggressive-behavior-video-classification/files/non_aggressive/3.mp4
/kaggle/input/aggressive-behavior-video-classification/files/non_aggressive/1.mp4
/kaggle/input/aggressive-behavior-video-classification/files/non_aggressive/4.mp4
/kaggle/input/aggressive-behavior-video-classification/files/non_aggressive/0.mp4
/kaggle/input/aggressive-behavior-video-classification/files/non_aggressive/2.mp4


**Preparing the data**

In [2]:
# installing required packages
!pip install git+https://github.com/sajjjadayobi/FaceLib.git
!git clone https://github.com/Pongpisit-Thanasutives/Variations-of-SFANet-for-Crowd-Counting

  Cloning https://github.com/sajjjadayobi/FaceLib.git to /tmp/pip-req-build-stki1ez9
  Running command git clone --filter=blob:none --quiet https://github.com/sajjjadayobi/FaceLib.git /tmp/pip-req-build-stki1ez9
  Resolved https://github.com/sajjjadayobi/FaceLib.git to commit 382841efec823f059f49754e5d378abb12cfb551
  Preparing metadata (setup.py) ... done
  Created wheel for facelib: filename=facelib-1.6-py3-none-any.whl size=44144 sha256=f541f816a9fbb3068e24ad75ea77789f6b29694137d758ace4442b23885d8f35
  Stored in directory: /tmp/pip-ephem-wheel-cache-_se7kfwf/wheels/54/7b/e1/15419c5fb2b8e868bddef5dbcbaf30c4901ee9078e228cd4b8
Successfully built facelib
Cloning into 'Variations-of-SFANet-for-Crowd-Counting'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 161 (delta 8), reused 29 (delta 6), pack-reused 130
Receiving objects: 100% (161/161), 14.04 MiB | 22.85 MiB/s, done.
Resolving del

In [3]:
# import required libraries
import cv2
from PIL import Image
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import torch
from torchvision import transforms

#import model
import os
os.chdir('/kaggle/working/Variations-of-SFANet-for-Crowd-Counting')
from models import M_SFANet_UCF_QNRF

# import facelib
from facelib import FaceDetector, AgeGenderEstimator

In [4]:
# convert the video to a series of images
def video_to_image (path, folder):
    global exp_fld
    
    # importing video
    vidcap = cv2.VideoCapture (path)
    exp_fld = folder
    
    # error handling
    try:
        if not os.path.exists (exp_fld):
            os.makedirs (exp_fld)
    except OSError:
        print ('Error: Creating directory of data')
    Count = 0
    sec = 0
    frameRate = 1 # secs of the video
    
    while (True):
        vidcap.set (cv2.CAP_PROP_POS_MSEC, sec * 1000)
        hasFrames, image = vidcap.read ()
        sec = sec + frameRate
        sec = round (sec, 2)
        
        # exporting the image
        if hasFrames:
            name = './' + exp_fld + '/frame' + str (Count) + '.jpg'
            cv2.imwrite (name, image)
            Count += 1
        else:
            break
    return print ('Image Exported')

# setting the path
path = ('/kaggle/input/aggressive-behavior-video-classification/files/aggressive')

# extracting images and storing for a first video
video_to_image ('/kaggle/input/aggressive-behavior-video-classification/files/aggressive/0.mp4', 'crowd')

# extracting images for forth video
video_to_image ('/kaggle/input/aggressive-behavior-video-classification/files/aggressive/3.mp4', 'movement')


Image Exported


KeyboardInterrupt: 

In [ ]:
# resizing the image
def img_re_sizing (dnst_mp, image):
    # normalizing
    dnst_mp = 255 * dnst_mp / np.max (dnst_mp)
    dnst_mp = dnst_mp [0][0]
    image = image [0]
    
    # empty image
    result_img = np.zeros ((dnst_mp.shape [0] * 2, dnst_mp.shape [1] * 2))
    
    # iterate for each image
    for i in range (result_img.shape [0]):
        for j in range (result_img.shape [1]):
            result_img [i][j] = dnst_mp [int (i / 2)]
            [int (j / 2)] /3
    result_img = result_img.astype (np.uint8, copy = False)
    
    # output
    return result_img

In [ ]:
# function to get a heatmap
def generate_dstys_map (o, dsty, cc, image_location):
    # define the fgr_imure
    fgr_im = plt.fgr_imure ()
    
    # define size
    col = 2
    rws = 1
    X = o
    
    # sum 
    add = int (np.sum (dsty))
    
    dsty = image_re_sizing (dsty, o)
    
    # adding original image and new generated heatmap image
    for i in range (1, col * rws + 1):
        # generate original image
        if i == 1:
            image = X
            fgr_im.add_subplot (rws, col, i)
            # setting axis
            plt.gca().set_axis_off ()
            plt.margins (0, 0)
            
            #locator
            plt.gca().xaxis.set_major_locator (plt.NullLocator ())
            plt.gca().yaxis.set_major_locator (plt.NullLocator ())
            # adjusting subplots
            plt.subplots_adjust (top = 1, bottom = 0, right = 1, left = 0,
                                hspace = 0, wspace = 0)
            # show image
            plt.imshow (image)
    # generate dstys image
    if i == 2:
        image = dsty
        fgr_im.add_subplot (rws, col, i)
        # setting axis
        plt.gca().set_axis_off ()
        plt.margins (0, 0)
        # locator
        plt.gca().xaxis.set_major_locator (plt.NullLocator ())
        plt.gca().yaxis.set_major_locator (plt.NullLocator ())
        # adjusting subplots
        plt.subplots_adjust (top = 1, bottom = 0, right = 1, left = 0,
                            hspace = 0, wspace = 0)
        # adding count
        plt.text (1, 80, 'M-SegNet * Est: '+ str (add)+',
                 Gt: '+str (cc), fontsize = 7, weight = "bold', color = 'w')
        # show image 
        plt.imshow (image), cmap = CM.jet
        
# image_nm with location
image_nm = image_location.split ('/')[-1]
image_nm = image_nm.replace ('.jpg', '_heatmap.png')

# saving the image
plt.savefgr_im (image_location.split (image_nm)
                [0]+'seg_'+ image_nm, transparent = True
                bbox_inches = 'tight', pad_inches = 0.0, dpi = 200)

In [ ]:
#  getting count of peoples
def get_count_people (image):
    # simple preprocessing
    trans  = transforms.Compose ([transforms,ToTensor (),
                                 transforms.Normalize ([0.485, 0.456, 0.406],
                                                      [0.229, 0.224, 0.225])])
    # sample image with height and width
    img = Image.open (image).convert ('RGB')
    height, width = img.size [1], img.size [0]
    height = round (height / 16) * 16
    width = round (width / 16) * 16
    
    # resize the image
    img_den = cv2.resize (np.array (img), (width, height), cv2.INTER_CUBIC)
    
    # transform 
    img = trans (Image.fromarray (img_den))[None, :]
    
    # lets define the model
    model = M_SFANet_UCF_QNRF.Model ()
    
    # load the model
    model.load_state_dict (torch.load ('/content/best_M-SFANet_UCF_QNRF.pth',
                                      map_location = torch.device ('cpu')))
    
    # evaluating the model
    model.eval ()
    dnst_mp = model (img)
    
    # final count 
    count = torch.sum (dnst_mp).item ()
    
    # return count, density and map
    return count, img_den, dnst_mp

In [ ]:
# importing the images
image_location = []
path_sets = ['/content/crowd']

# loading all the images
for path in path_sets:
    for img_path in glob.glob (os.path.join (path, '*.jpg')):
        image_location.append (img_path)
image_location [:3]

In [ ]:
# getting the count of people by image
# define empty list
list_df = []

# loop through each image
for i in image_location:
    count, img_den, dnst_mp = get_count_people (i)
    generate_dens_map (img_den, dnst_mp.cpu().detach ().numpy(), 0, i)
    list_df = list_df + [[i, count]]
    
# create the dataframe with image id and count
df = pd.DataFrame (list_df, columns = ['image', 'count'])

# sort and show
df.sort_values (['image']).head ()

df.describe()

In [ ]:
import seaborn as sns
sns.histplot (data = df['count'])

In [ ]:
# getting all the images from the path
image_location = []
path_sets = ['content/movement']

# loading all images
for path in path_sets:
    for img_path in glob.glob (os.path.join (path, '*.jpg')):
        image_location.append (img_path)

image_location [:3]

# getting the count of people by image
list_df = []

In [ ]:
# checking if there is any movement for each frame in video
for i in image_location:
    count, img_den, dnst_mp = get_count_people (i)
    generate_dens_map (img_den, dnst_mp.cpu().detach().numpy(), o, i)
    list_df = list_df + [[i, count]]
    
# detecting the data into dataframe
detected_df = pd.DataFrame (list_df, columns = ['image', 'count'])
detected_df ['movement'] = np.where (detected_df ['count'] > 3, 'yes', 'no')
detected_df.filter (items = [45, 56, 53], axis = 0)

In [ ]:
# print the image where movement = 'yes'
image = '/content/movement/frame25.jpg'
Image.open (image)

image = '/content/movement.frame26.jpg'
Image.open (image)

In [ ]:
# identify age and gender
face_detector = FaceDetector ()
age_gender_detector = AgeGenderEstimator ()

# reading image
img = '/content/movement.frame1.jpg'
image = cv2.imread (img, cv2.IMREAD_UNCHANGED)
Image.open (img)

In [ ]:
# checking the gender 
faces, boxes, scores, landmarks = face_detector.detect_align (image)
genders, ages = age_gender_detector.detect (faces)
print (genders, ages)